# Particle Filter

## Code for robot spawn, move and sense

In [7]:
# In this exercise, try to write a program that
# will resample particles according to their weights.
# Particles with higher weights should be sampled
# more frequently (in proportion to their weight).

# Don't modify anything below. Please scroll to the 
# bottom to enter your code.

from math import *
import random

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError('Robot cant move backwards')      
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

## Task 1

In this case, all noise parameters are 0

In [31]:
# starts at 30.0, 50.0, heading north (=pi/2)
# turns clockwise by pi/2, moves 15 meters
# senses
# turns clockwise by pi/2, moves 10 meters
# senses


In [33]:
my_robot = robot()
my_robot.set(30.,50.,pi/2)
my_robot = my_robot.move(-pi/2, 15)
print(f"Landmarks at: {my_robot.sense()}")
my_robot = my_robot.move(-pi/2, 10)
print(f"Landmarks at: {my_robot.sense()}")
print(f"Robot at {my_robot}")

Landmarks at: [39.05124837953327, 46.09772228646444, 39.05124837953327, 46.09772228646444]
Landmarks at: [32.01562118716424, 53.150729063673246, 47.16990566028302, 40.311288741492746]
Robot at [x=45.0 y=40.0 orient=4.7123]


## Task 2
Adding noise parameters

In [49]:
# forward_noise = 5.0, turn_noise = 0.1, sense_noise = 5.0
# rest is same as task 1
# starts at 30.0, 50.0, heading north (=pi/2)
# turns clockwise by pi/2, moves 15 meters
# senses
# turns clockwise by pi/2, moves 10 meters
# senses

In [50]:
f_noise = 5.0
t_noise = 0.1
s_noise = 5.0

In [51]:
my_robot = robot()
my_robot.set(30.,50.,pi/2)
# adding noise parameters
my_robot.set_noise(f_noise, t_noise, s_noise)
my_robot = my_robot.move(-pi/2, 15)
print(f"Landmarks at: {my_robot.sense()}")
my_robot = my_robot.move(-pi/2, 10)
print(f"Landmarks at: {my_robot.sense()}")
print(f"Robot at {my_robot}")

Landmarks at: [43.18807262386186, 54.170935853039126, 35.66383978464553, 39.28931584486206]
Landmarks at: [33.82274656099736, 53.19813665763713, 48.99281850237137, 36.94686968015874]
Robot at [x=46.400 y=38.196 orient=4.7963]


## Create particles

In [63]:
N = 1000
# create n particles

particles = [robot() for i in range(N)]

len(particles)

1000

## Move particles

In [64]:
# turn by 0.1 , and move by 5
p_moved = [x.move(0.1, 5) for x in particles]

In [65]:
particles[0], p_moved[0]

([x=49.641 y=82.278 orient=0.6657], [x=53.245 y=85.743 orient=0.7657])